# 1. 데이터 로드

In [1]:
import pandas as pd

In [2]:
test_df = pd.read_csv('/content/drive/MyDrive/MLDL_개인프로젝트/2024/Sentiment_Classification/inference.csv', index_col=0)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16984 entries, 0 to 16983
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RawText          16984 non-null  object 
 1   GeneralPolarity  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 398.1+ KB


In [3]:
test_df.head()

,RawText,GeneralPolarity
0,솔직히 타 쇼핑몰에서 엄마상품 시켜드렸는데 매번 실패했거든요 이 상품은 채널 돌리다...,NaN
1,다크는 다른바지에 비해 뻣뻣한 편이네요 촉감이.. 세벌 모두 쭉쭉 잘 늘어나고 착용...,NaN
2,OOO 방송하길래 예쁘서 구입했는데 32를 주문했는데 너무 크더라구요 ㅋㅋ 30으로...,NaN
3,홈쇼핑에서 구매를 많이 하는편인데 OOO여성 매직핏 대님은 진짜 잘 늘어나고 사이즈...,NaN
4,OOO 겨윤 지날때까지 기다렸어요 디자인 예쁘고 살이 많이 쪄서 크게 주문했...,NaN


# 2. 텍스트 전처리

In [4]:
import re

def remove_special_characters(text):

    # 정규 표현식을 사용하여 특수문자를 제거
    # [^가-힣0]는 한글만 남기고 나머지는 제거
    clean_text = re.sub(r'[^가-힣\s]', '', text).strip()
    return clean_text

In [5]:
!pip install soynlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 7.9 MB/s eta 0:00:00


In [6]:
from soynlp.normalizer import *
def normalize_repeat_characters(text):
    clear_output = repeat_normalize(text, num_repeats=2)
    return clear_output

In [7]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2024-09-19 09:21:55--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1 [following]
--2024-09-19 09:21:55--  https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc26c55d9f899186c17d83379f88.dl.dropboxusercontent.com/cd/0/inline/Ca1W2pud2gym8kwOVNJcwIN2rVu2XtsxmHVzmzHTdormq2zR5cp3_21KvMOP2x1Esp0m6lu-QJaAfEAWW8_uGVCmjDg9d8KmFWXuBichYTaBiIj8eAWzPw2xd455qInBYX86-AcjdmFe6en9qmLIbcmG/file?dl=1# [following]
--2024-09-19 09:21:56--  https://

In [8]:
from konlpy.tag import Okt

okt = Okt()

# 관심 있는 품사 태그 목록
significant_tags = ['Noun', 'Verb', 'Adjective', 'Adverb']

# 중요 품사만 추출
def pos_text(corpus):

    pos_tagged = ''

    # Okt 형태소 분석기로 형태소 분석 및 품사 태깅
    # stem=True 옵션으로 원형 변환
    for morph, tag in okt.pos(corpus, stem=True):

        # 관심 있는 품사만 추출
        if tag in significant_tags:
            pos_tagged += morph + ' '

    return pos_tagged

In [9]:
# 파일을 읽기 모드로 열기
with open('/content/drive/MyDrive/MLDL_개인프로젝트/2024/Sentiment_Classification/stopword.txt', 'r', encoding='utf-8') as file:
    stopwords = file.readlines()  # 파일의 모든 줄을 리스트로 읽기

stopwords = [word.strip() for word in stopwords]

def remove_stopwords(corpus):
    filtered_corpus = []
    for text in corpus:
        words = text.split()
        filtered_words = [word for word in words if word not in stopwords]
        filtered_text = ' '.join(filtered_words)
        filtered_corpus.append(filtered_text)

    return filtered_corpus

In [10]:
!pip install transformers

In [11]:
from transformers import BertTokenizer

def embedding_corpus(corpus):

    # BERT Tokenizer를 이용한 text embedding 수행
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # 토크나이저를 이용해 텍스트를 토큰화하고 인덱스로 변환
    TOKEN_MAX_LENGTH = 128
    tokenized_inputs = tokenizer(corpus, padding=True, truncation=True, max_length=TOKEN_MAX_LENGTH, return_tensors="pt")

    return tokenizer, tokenized_inputs

In [12]:
def preprocess_text(corpus):

    corpus = remove_special_characters(corpus)
    corpus = normalize_repeat_characters(corpus)
    corpus = pos_text(corpus)
    corpus = remove_stopwords(corpus)
    tokenizer, input_data = embedding_corpus(corpus)

    return tokenizer, input_data

# 3. Model load

In [14]:
import torch
import torch.nn as nn

In [15]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 3.1) Vanilla RNN

In [18]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):

        super(SentimentRNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)  # 임베딩 레이어
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)  # 바닐라 RNN 레이어
        self.fc = nn.Linear(hidden_dim, output_dim)  # Fully Connected 레이어

    def forward(self, x):

        embedded = self.embedding(x)  # 텍스트 임베딩
        rnn_out, _ = self.rnn(embedded)  # RNN 처리
        output = rnn_out[:, -1, :]  # 마지막 타임스텝의 은닉 상태를 가져옴
        output = self.fc(output)  # 분류를 위한 출력

        return output

# 하이퍼파라미터 설정
vocab_size = tokenizer.vocab_size
embedding_dim = 100  # 임베딩 차원
hidden_dim = 128     # RNN 은닉 상태 차원
output_dim = 3       # 긍정, 부정, 중립 3개의 클래스

# 모델 초기화
model = SentimentRNN(vocab_size=len(tokenizer), embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=output_dim).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/MLDL_개인프로젝트/2024/Sentiment_Classification/Vanilla_RNN_model.pt', map_location=device))

model

SentimentRNN(
  (embedding): Embedding(30522, 100)
  (rnn): RNN(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=3, bias=True)
)

In [75]:
import random

label_dic = {0 : '부정', 1 : '중립', 2 : '긍정'}

def predict_sentiment(model, tokenizer, random_idx):

    sample_text = test_df.loc[random_idx, 'RawText']

    tokenizer, input = preprocess_text(sample_text)
    input_data = input['input_ids'].to(device)

    model.eval()

    with torch.no_grad():
        outputs = model(input_data)

    # 가장 높은 확률을 가진 클래스 선택
    predicted_class = torch.softmax(outputs, dim=1) # shape = (56, 3)
    predicted_class = torch.argmax(predicted_class, dim=1)
    predicted_class = label_dic[torch.max(predicted_class).item()]

    return sample_text, predicted_class

random_idx = random.randint(0, len(test_df) - 1)

sample_text, predicted_class = predict_sentiment(model, tokenizer, random_idx)
print(f'원본 텍스트: {sample_text} \n분류 결과: {predicted_class}')

원본 텍스트: 화질이 나쁜건지 화면이 선명하지가 않아 별로네요 그래도 가격대비해 화각도 넓고 보기 좋아요 
분류 결과: 긍정
